# Data Handling With Pandas

## Introduction
Pandas is built on numpy and can be used to interepet data. This notebook is used to maintain the collection of all useful tasks that can be performed with Pandas

## Setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

file_path = 'data/example_data/data.csv'
data = pd.read_csv(file_path)
data

## Basic Inference and Data Access

In [ ]:
# show first few rows
data.head(n=2) # default is 5

In [ ]:
# Describe on csv data gives count, mean, variance and other statistical details about each parameter in the input 
data.describe()

In [ ]:
# List parameters(columns)
data.columns

In [ ]:
# gives the data type of each input parameter
data.dtypes

In [ ]:
#Added info on input parameters
data.info

In [ ]:
# accessing data by parameter
data['Customer Number']

In [ ]:
# getting statistical values by paramter
data.Year.mean()

## Data Conversion

In [ ]:
# conversion using astype
data['Customer Number'].astype('int')

In [ ]:
# doesn't work as the values(like 15,000.00$) are not stright forward integeres
try:
    data['2016'].astype('float')
except Exception as e:
    print(e)

In [ ]:
# doesn't work as one of the values is 'Closed' (not a number)
try:
    data['Jan Units'].astype('int')
except Exception as  e:
    print(e)

In [ ]:
# doesn't work as expected as it can't interpret from Y/N
data['Active'].astype('bool')

In [ ]:
# using to_numeric to convert to int
# coerce sets errored instance to NaN
pd.to_numeric(data['Jan Units'], errors='coerce')

In [ ]:
# setting errored values to zero
pd.to_numeric(data['Jan Units'], errors='coerce').fillna(0)

In [ ]:
# using to_datatime to generate date from individual parameters
pd.to_datetime(data[['Month', 'Day', 'Year']])

## Other Useful Tools

### apply

In [ ]:
# Using a convertor function and apply to change currency to int
def convertor(value):
    result = value.replace(',','').replace('$','')
    return float(result)

data['2016'].apply(convertor)

In [ ]:
# Using lambdas and apply to change currency to int
data['2016'].apply(lambda x : x.replace(',','').replace('$','')).astype('float')

### np.where

In [ ]:
# where acts like if clause
np.where(data['Active'] == "Y", True, False)

### dropna

In [ ]:
# dropna is used to drop rows or columns with na.
# axis = 0 - rows with na are dropped
# axis = 1 - columns with na are dropped
pd.to_numeric(data['Jan Units'], errors='coerce').dropna(axis=0)

### Selecting columns

In [ ]:
# this cell shows how to pick few necessary columns from input data
data[['Customer Number', '2016', 'Active']]

### Creating new DataFrame

In [ ]:
# this way a new dataframe can be initialized.
result = pd.DataFrame({'Number': data['Customer Number'],
                     'Active' : data['Active']})

### Output to CSV 

In [ ]:
# A dataframe can be output to csvfile
result.to_csv('output.csv', index=False)

## Customizing data import

In [ ]:
data2 = pd.read_csv(file_path,
                   dtype={'Customer Number': 'int'},
                   converters={'2016': convertor,
                               '2017': convertor,
                               'Percent Growth': lambda x : float(x.replace('%',''))/100,
                               'Jan Units': lambda x: pd.to_numeric(x, errors='coerce'),
                               'Active': lambda x: np.where(x == "Y", True, False)
                              })

data2 = data2[['Customer Number', '2016', '2017', 'Percent Growth', 'Jan Units']]
data2

## Dealing with Missing Values
The data we work on generally comes with lot of missing values. The models we work with do not accommodate data with missing values. 

### Removing rows with missing values
Removing rows with NaN or blanks is straight forward approach. If there are very few missing fields it can be simpler to use this with out loosing much of information.

In [ ]:
#As mentioned above dropna can be for this
data2.dropna(axis=0)

### Removing columns with missing values
Removing columns with missing values is much costlier as we would be forced to ignore a feature with missing values. This will be too costly if the feature plays strong role in prediction of target.

In [ ]:
# we are again going to use dropna to drop columns.
data2.dropna(axis=1)

### Imputation
Imputation is replacing the missing values with simulated ones. The common pattern is to replace the missing value with average.
Imputation is frequently used as it wouldn't result in discarding of data. There are frequent cases where models performed better with imputed values.

In [ ]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer()
my_imputer.fit_transform(data2)

### Imputation with indicator
To better imputation is to relay the information about imputed values to model. Many models by scikit learn that support this. In many cases this helps models to be vary of making judgement based on imputed values.
This generally done by adding new columns 

In [ ]:
# Copy the values to new matrix
data_copy = data2.copy()

# Collect column names with missing values
columns_with_missing_values = (col for col in data_copy.columns if data_copy[col].isnull().any())

# Now for these colums create new columns which stores boolean values to indicate missing value.
for col in columns_with_missing_values:
    data_copy[col+'_was_missing'] = data_copy[col].isnull()
    
data_copy
    
# Now impute the missing values
#my_imputer = Imputer()
#my_imputer.fit_transform(data_copy)

## Feature Encodings
Machine learning models aren't built to work on categorical data. We need to modify the input categorical data to match the input constraints of these models. One of the well known method is One Shot Encoding

### One Shot Encoding
One shot encoding is converting categorical values for feature to a set of boolean feature for each category. Say we have colors as feature then we will be converting it to multiple new feature as is_orange, is_red etc. 
As you have seen this isn't scalable but works. If category count is huge we just push all the minorities to a feature like 'is_others'.

In [ ]:
# pandas get_dummies converts catogorical text to features with boolean values.
pd.get_dummies(data[['Customer Number', 'Active']])

# we need to have same ordering of columns in test and training data. can be done with align

# join - left : preserves columns from left side and remove ones specific to right

## Sources
* http://pbpython.com/pandas_dtypes.html
* https://www.kaggle.com/dansbecker/explore-your-data
* https://www.kaggle.com/dansbecker/your-first-machine-learning-model
